In [1]:
using NBInclude
@nbinclude("datastructs.ipynb")

combine_ge (generic function with 1 method)

In [2]:
# process adj list 
# create dictionary of gene ID --> index/location of gene's head and tail 

function process_adj_list_helper(ge::GeneEnd, idx::Ref{Int}, gid_to_loc::DefaultDict{Int, Vector{Int}})
    if ge.gene == Telomere() 
        return
    end  

    if ge.head == true
        gid_to_loc[ge.gene.id][2] = idx[]
    else 
        gid_to_loc[ge.gene.id][1] = idx[]
    end
end 

function process_adj_list(adj_list:: Vector{Adjacency})
    geneid_to_location = DefaultDict{Int, Vector{Int}}(() -> zeros(Int, 2)) # tail = idx 1, head = 2 in array
    idx = Ref{Int}(1)

    for adj in adj_list
        process_adj_list_helper(adj.left, idx, geneid_to_location)
        process_adj_list_helper(adj.right, idx, geneid_to_location)
        idx[] += 1
    end 
    
    return geneid_to_location 
end 
     

process_adj_list (generic function with 1 method)

In [3]:
# helpers for dcj operations and distance

#TODO combine with find_next_adj() cuz they do the same thing 
# finds target gene end in source adjacency list 
# returns index and left/right in the source adj list

# e.g., given adj set: {a:h, b:t} ... {b:h, d:t} & ge = b:t, 
# finds {b:h, d:t}, returns d:t 
function find_tar_ge_in_src_adjs(target_ge::GeneEnd, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adjs::Vector{Adjacency})
    (target_ge.head == true) ? th_idx = 2 : th_idx = 1
        
    # print(target_ge.gene)
    gene_id = target_ge.gene.id
    # print(gene_id)

    src_ge_idx = src_gid_to_l[gene_id][th_idx]

    adj = src_adjs[src_ge_idx]

    if adj.left.gene == Telomere() 
        return adj.left, src_ge_idx, adj
    elseif adj.right.gene == Telomere() 
        return adj.right, src_ge_idx, adj
    end 

    if adj.left.gene.id == gene_id
        src_ge = adj.left
    else 
        src_ge = adj.right 
    end 
    
    return src_ge, src_ge_idx, adj
end 


function update_adj_set(p::GeneEnd, q::GeneEnd, u::GeneEnd, v::GeneEnd, u_idx::Int, v_idx::Int, src_adjs::Vector{Adjacency}, src_adj_set::Set{Adjacency}) 
    #  replace adj u and v in A by ( {p, q} and (u\{p}) U (v\{q}) )
    pq = combine_ge(p, u_idx, q, v_idx)
    other_ge_u = other_adjacency_end(u, src_adjs[u_idx])
    other_ge_v = other_adjacency_end(v, src_adjs[v_idx])
    excluding_pq = combine_ge(other_ge_u, u_idx, other_ge_v, v_idx) 

    #  remove adj with u, v in src_adj_set 
    delete!(src_adj_set, src_adjs[u_idx])
    delete!(src_adj_set, src_adjs[v_idx])
    
    #  add adjusted adjacencies to src_adj_set
    push!(src_adj_set, pq)
    push!(src_adj_set, excluding_pq)

    return pq, excluding_pq
end 



update_adj_set (generic function with 1 method)